# Playing with the gensim coherence measures

## Introduction

Testing out difference 'coherence' pipeline combinations.
Topic models: LDA, NNMF, LSA, PCA + varimax

In [105]:
%load_ext autoreload
%autoreload 2
import logging
import warnings
import numpy as np
import pandas as pd
import io
from process_topics import show_topic_words, run_all, coherence_widget, NewCoherence
from gensim.models import CoherenceModel, LdaModel, HdpModel, nmf, LdaMulticore
from gensim.corpora import Dictionary, csvcorpus
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import *
from time import time
warnings.filterwarnings('ignore')  # To ignore all warnings that arise here to enhance clarity
from ipywidgets import interact, interactive, IntSlider, Layout, interact_manual, fixed, interactive_output, FloatSlider
import ipywidgets as widgets
import qgrid
import logging

#logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload



### Set up corpus

Set up test corpus, should be tokenized, sotpwords removed

In [89]:
%%time
# my_csv = pd.read_csv('..\..\wiki_movie_plots_deduped.csv')
# plots = my_csv['Plot']
elections = io.open('Election2008Paragraphes.txt',encoding = "ISO-8859-1")
electionlines  =elections.readlines()

CUSTOM_FILTERS = [lambda x: x.lower(),  strip_punctuation, strip_multiple_whitespaces, strip_numeric,remove_stopwords, strip_short]

texts = [preprocess_string(line, filters=CUSTOM_FILTERS) for line in electionlines]


dictionary = Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]
data = [texts, dictionary, corpus]


CPU times: user 3.51 s, sys: 74.6 ms, total: 3.59 s
Wall time: 3.59 s


In [3]:
#changes made to FARotate in the code will require kernel restart if reloading this cell!
m = coherence_widget(data)
display(m)


aW50ZXJhY3RpdmUoY2hpbGRyZW49KFJhZGlvQnV0dG9ucyhkZXNjcmlwdGlvbj11J0Nob29zZSBNb2RlbCcsIGxheW91dD1MYXlvdXQod2lkdGg9dScyNTBweCcpLCBvcHRpb25zPSgnTk1GJyzigKY=


In [5]:
topics, coherences, both= m.result

In [6]:
pd.DataFrame(topics)

,Topic
0,care health insurance plan costs quality americans families companies veterans
1,america world people today global energy nation future work like
2,need energy world oil security change national nuclear military time
3,people know american want going think good said got like
4,government american federal people security economy way time trust spending
5,iraq troops military war iran president qaeda afghanistan security plan
6,president country want states believe time united going change thank
7,war time world years like bush end going said americans
8,tax jobs families help taxes workers americans working middle economy
9,new energy jobs create nuclear york years like time use


In [92]:
from __future__ import division
from process_topics import coherence_scores
from similarity import *
style = {'width': '1000px', 'height': '30px'}

w=IntSlider(10,0,1000, description = 'w',layout=style)
w2=IntSlider(10,0,1000, description = 'w2',layout=style) 
co=IntSlider(10,0,1000, description = 'co',layout=style)
ndocs=IntSlider(1000,0,10000, description = 'ndocs',layout=style)
print co.value
exp = FloatSlider(value=0.0, min=0, max=20, description='exp', layout=style)
joint = FloatSlider(value=0.0, min=0, max=20, description='exp', layout=style)

exp.style.handle_color= 'yellow'
#print widgets.__dict__
ui = widgets.VBox([w, w2, co, ndocs,exp])

    
def similarities(w,w2,co,ndocs, held=[]):
    scores = calculate_sims(w,w2,co,ndocs,'all')
    #held.append(w)
    display(pd.DataFrame(scores))
    return scores

out =interactive_output(similarities,
         {'w':w,'w2':w2,'co':co,'ndocs':ndocs})

out.layout.height= '500px'
display(ui,out)

@out.capture()
def set_exp_per_rest(change):
    print (w.value+ w2.value)/ ndocs.value
    exp.value = ((w.value+ w2.value)/ ndocs.value) *100
    print exp.value 
    
w.observe(set_exp_per_rest, names="value")
w2.observe(set_exp_per_rest, names="value")
co.observe(set_exp_per_rest, names="value")
ndocs.observe(set_exp_per_rest, names="value")


    

10


VkJveChjaGlsZHJlbj0oSW50U2xpZGVyKHZhbHVlPTEwLCBkZXNjcmlwdGlvbj11J3cnLCBsYXlvdXQ9TGF5b3V0KGhlaWdodD11JzMwcHgnLCB3aWR0aD11JzEwMDBweCcpLCBtYXg9MTAwMCnigKY=


Output(layout=Layout(height=u'500px'))

In [108]:
np.set_printoptions(threshold=np.inf, linewidth=1000)

pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1500)
pd.set_option('display.column_space', 2)

texts2 = [preprocess_string(line, filters=CUSTOM_FILTERS) for line in e[1:5]]
for tex in texts2:
    print ' '.join(tex)
tf_vectorizer = CountVectorizer()
tftexts = [' '.join(text) for text in texts2]
tf = tf_vectorizer.fit_transform(tftexts)
tf[tf>1]=1

xc = (tf.T * tf)
print 'occur'
print tf.toarray()
print 'cooccur'
tf_feature_names = tf_vectorizer.get_feature_names()
for i,name in enumerate(tf_feature_names):
    width = 12
    spacer =width - (len(str(i)) + len(name))
    if i-1%10==0 or i==0:
        print i,name,' '*spacer,
    else:
        print i,name,' '*spacer,
print ''
cooccur =xc.todense()
for i,x in enumerate(cooccur):
    print tf_feature_names[i],tf_feature_names[i+i]
    print float(cosine_similarity(x,cooccur[i+1])), float(cosine_distances(x,cooccur[i+1])), 'eucl ', float(euclidean_distances(x,cooccur[i+1]))
display(pd.DataFrame(cooccur))



 year democratic unionists face historic choice share political power nationalist community
year meet hope mother prayer taken government hands ireland road permanent peace
yeats said long sacrifice stone heart wait hearts turn stone challenge political leaders follow lincoln called better angels nature century look forward write new chapter irish history
want thank honor know mother loved adored awards given makes feel totally worthy mother eyes
occur
[[0 0 0 0 0 0 0 0 1 1 1 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 1 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 1 1 0 0 0 1 1 0 0 1 1 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0]
 [0 1 0 1 1 1 1 1 0 0 0 0 0 0 1 1 0 0 0 1 1 0 1 0 0 0 1 0 1 1 1 1 0 0 0 0 0 1 1 0 0 1 0 0 0 1 1 0 1 0 0 0 1 0 1 0 0 1 0 1]
 [1 0 1 0 0 0 0 0 0 0 0 1 0 1 0 0 1 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 1 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 1 1 0 0 0]]
cooccur
0 adored       1 angels       2 awards    

IndexError: list index out of range